# 2022 Capstone Project 
# Where can soil moisture improve rainfall-triggered landslide predictability?
# Team: Jacquelyn Witte

In [11]:
import os

import earthpy as et
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from geopandas import GeoDataFrame
from shapely.geometry import Point

import warnings
warnings.filterwarnings('ignore')

sns.set(font_scale=1.2, style="whitegrid")

## Why do we care about landslides?


Landslides have significant economic impacts. There can be loss of life, wildlife and wildlife habitat. Landslides are a serious geologic hazard common to many areas in the US and Canada. In Colorado, which is where I live, a high percentage of our population lives in the Rockies. So transportation and infrastructure are impacted as you can see from the CNN caption below of a mudslide event on the I-70 which is our main highway artery across the Rockies.

<div>
<img src="images/cnn_newsarticle_glenspringsLandslideCO.png " width="500"/>
</div>



On another note, my husband I were driving home from the Oregon coast during an Atmospheric river event and there were crews along the valley roads building retaining walls to prevent landslides because of the soil erosion caused by wildfires in that region. So climate change is also factor and one which confounds our ability to predict rainfall trigger landslides. Climate change is a chaotic system and the basis of all models is pattern and order. 

This is where soil moisture and composition and knowing the history of the area and land-uses come into play in this project. 

## Motivation

Landslides have multiple causes:  rainfall, snowmelt, erosion, earthquakes, volcanoes, disturbance by human activities, or any combination of these and other factors. This study focuses on rainfall-triggered landslides. Predicting them with a high degree of accuracy to save lives and maintain infrastructure is challenging. Precipitation data sources are heterogeneous - radar, satellite, in-situ and ensemble model predictions using available precipitation products carry large uncertainties (https://onlinelibrary.wiley.com/doi/full/10.1002/hyp.14260). This make it challenging to predict the timing, severity, extent, and even likelihood that a landslide will occur. 

Soil moisture can be a potential indicator of the type of rainfall induced landslide. We can use soil moisture data to group landslides into two types (1) shallow slope failures - saturation induced by rainfall infiltration and (2) run off driven landslides - triggered by intense storms.  This informs the forecaster which models and precipitation products are optimal to use to better predict a landslide event.

The purpose of this study is to address the need to improve our understanding of landslides caused by rainfall. To seek answers to the question of where and when will landslides occur?

This work follows on research done by Elsa Culler and colleagues in their paper:
> Culler, E. S., Badger, A. M., Minear, J. T., Tiampo, K. F., Zeigler, S. D., & Livneh, B. (2021). A multi-sensor evaluation of precipitation uncertainty for landslide-triggering storm events. Hydrological Processes, 35( 7), e14260. https://doi.org/10.1002/hyp.14260

## Study region: Colorado, USA

The study area will be Colorado, because I live here and am somewhat familiar with the landscape. 

Towns and cities in Colorado's mountainous or hilly terrain are growing and spreading as populations move into these popular destination areas. It is important to understand the potential exposure to landslide hazards, and how counties can maintain optimal land-use and infrastructure to reduce the costs of living with landslides. 

Below is map of the Colorado with all the Landslide events between 2007-2016. The locations are color coated by trigger - Continuous rain, downpour, rain, tropical cyclone which we have one documented and unknown.
> Data Source: Global Landslide Catalog https://doi.org/10.1002/hyp.14260

In [16]:
# Change to data directory
data_dir = os.path.join(et.io.HOME,
                        'earth-analytics',
                        'data')
os.chdir(data_dir)
# Read Global Landslide Catalog CSV to a dataFrame
glc = pd.read_csv(os.path.join('capstone', 
                               'landslide', 
                               'nasa.glc.export.csv'))
# Subset to Colorado and remove non-rainfall events
glc_co = glc[(glc['country_code'] == 'US') & (
                                glc['admin_division_name'] == 'Colorado') & (
                                glc['landslide_trigger'] != 'freeze_thaw') &(
                            glc['landslide_trigger'] != 'snowfall_snowmelt')
            ]
# covert to datetime
glc_co['date'] = pd.to_datetime(glc_co['event_date'])
glc_co = glc_co.set_index('date').sort_index()

# georeference the locations to plot on Folium
geometry = [Point(xy) for xy in zip(glc_co['longitude'], glc_co['latitude'])]
crs = 'epsg:4326'
glc_co_epsg4326 = GeoDataFrame(glc_co, crs=crs, geometry=geometry)

# Create a symbology dictionary that maps "type" to color. 
triggers = np.unique(glc_co_epsg4326['landslide_trigger'])
colors = ['red', 'green', 'blue', 'orange','lightgray']
points_symb = dict(zip(triggers,colors))

# map the Landslide events
# Create Title
loc = 'Rainfall-triggered Landslides in Colorado (2007 - 2017)'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m2 = folium.Map([39.0, -105.27], zoom_start=7, tiles='Stamen Terrain')

for i, loc in glc_co_epsg4326.iterrows():
    folium.Marker(location=[loc['latitude'], loc['longitude']],
                  icon=folium.Icon(color=points_symb[loc['landslide_trigger']],
                                   prefix='glyphicon', icon='off')).add_to(m2)

# Continuous Rain
folium.CircleMarker(location=[40.5, -103], radius=10,
                    color='red',
                    fill_color='red',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[40.5, -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Continuous Rain'}</b>""")
).add_to(m2)

# Downpour
folium.CircleMarker(location=[40, -103], radius=10,
                    color='green',
                    fill_color='green',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[40, -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Downpour'}</b>""")
).add_to(m2)

# Rain
folium.CircleMarker(location=[39.5, -103], radius=10,
                    color='blue',
                    fill_color='blue',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[39.5, -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Rain'}</b>""")
).add_to(m2)
   
    
# Tropical Cyclone
folium.CircleMarker(location=[39., -103], radius=10,
                    color='orange',
                    fill_color='orange',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[39., -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Tropical Cyclone'}</b>""")
).add_to(m2)
   
# Unknown
folium.CircleMarker(location=[38.5, -103], radius=10,
                    color='lightgray',
                    fill_color='lightgray',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[38.5, -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Unknown'}</b>""")
).add_to(m2)

    
#Add Title
m2.get_root().html.add_child(folium.Element(title_html))
m2

## Data Sources

### [NASA Global Landslide Catalog (2007-2016)](https://data.nasa.gov/Earth-Science/Global-Landslide-Catalog/h9d8-neg4)
> Kirschbaum, D. B., Adler, R., Hong, Y., Hill, S., & Lerner-Lam, A. (2010). A global landslide catalog for hazard applications: Method, results, and limitations. Natural Hazards, 52(3), 561–575. https://doi.org/10.1007/s11069-009-9401-4

The Global Landslide Catalog (GLC) was developed with the goal of identifying rainfall-triggered landslide events around the world, regardless of size, impacts or location. The GLC considers all types of mass movements triggered by rainfall, which have been reported in the media, disaster databases, scientific reports, or other sources. The GLC has been compiled since 2007 at NASA Goddard Space Flight Center.

Elsa Culler (CIRES/CU Boulder) and colleagues provided a subset of landslide locations verified with ancillary satellite imagery.

### [Groundwater and Soil Moisture Conditions from GRACE Data Assimilation L4 7-days 0.125 x 0.125 degree V4.0 (GRACEDADM_CLSM0125US_7D)](https://disc.gsfc.nasa.gov/datasets/GRACEDADM_CLSM0125US_7D_4.0/summary?keywords=GRACE)

> Matthew Rodell and Hiroko Kato Beaudoing, NASA/GSFC/HSL (2021),
Groundwater and Soil Moisture Conditions from GRACE Data Assimilation L4 7-days 0.125 x
0.125 degree Version 4.0, Greenbelt, Maryland, USA: Goddard Earth Sciences Data and
Information Services Center (GES DISC), Accessed 22 April 2022, 10.5067/UH653SEZR9VQ.

- Shortname:GRACEDADM_CLSM0125US_7D
- Longname:Groundwater and Soil Moisture Conditions from GRACE Data Assimilation L4 7-days 0.125 x 0.125 degree V4.0
- DOI:10.5067/UH653SEZR9VQ
- Version:4.0
- Format:netCDF
- Spatial Coverage:-125.0,25.0,-67.0,53.0
- Temporal Coverage:2002-04-01 to  2022-01-02
- File Size:1 MB per file
- Data Resolution
- Spatial:0.125 ° x 0.125 °
- Temporal:7 days

### [SMAP L3 Radiometer Global Daily 36 km EASE-Grid Soil Moisture, Version 7](https://nsidc.org/data/SPL3SMP/versions/7)
> O'Neill, P. E., S. Chan, E. G. Njoku, T. Jackson, R. Bindlish, and J. Chaubell. 2020. SMAP L3 Radiometer Global Daily 36 km EASE-Grid Soil Moisture, Version 7. [Indicate subset used]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center, Accessed 22 April 2022, doi: https://doi.org/10.5067/HH4SZ2PXSP6A. 

This Level-3 (L3) soil moisture product provides a composite of daily estimates of global land surface conditions retrieved by the Soil Moisture Active Passive (SMAP) passive microwave radiometer. SMAP L-band soil moisture data are resampled to a global, cylindrical 36 km Equal-Area Scalable Earth Grid, Version 2.0 (EASE-Grid 2.0).

### [GPM IMERG Late Precipitation L3 1 day 0.1 degree x 0.1 degree V06 (GPM_3IMERGDL)](https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGDL_06/summary?keywords=%22IMERG%20late%22)
> Huffman, G.J., E.F. Stocker, D.T. Bolvin, E.J. Nelkin, Jackson Tan (2019), GPM IMERG Late Precipitation L3 1 day 0.1 degree x 0.1 degree V06, Edited by Andrey Savtchenko, Greenbelt, MD, Goddard Earth Sciences Data and Information Services Center (GES DISC), Accessed: 18 April 2022, 10.5067/GPM/IMERGDL/DAY/06

- Shortname:GPM_3IMERGDL
- Longname:GPM IMERG Late Precipitation L3 1 day 0.1 degree x 0.1 degree V06
- DOI:10.5067/GPM/IMERGDL/DAY/06
- Version:06
- Format:netCDF
- Spatial Coverage:-180.0,-90.0,180.0,90.0
- Temporal Coverage:2000-06-01 to  2022-04-20
- File Size:12 MB per file
- Data Resolution
- Spatial:0.1 ° x 0.1 °
- Temporal:1 day

### [ESA Climate Change Initiative soil moisture version 03.3](https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-soil-moisture?tab=overview)
> R. van der Schalie, R. De Jeu, C. Paulik, W. Dorigo, C. Reimer, R. Kidd, 2017. C3S D312a
Lot 7.Algorithm Theoretical Basis Document (ATBD): Soil Moisture

> Dorigo, W.A., Gruber, A., De Jeu, R.A.M., Wagner, W., Stacke, T., Loew, A.,
Albergel, C., Brocca, L., Chung, D., Parinussa, R.M., Kidd, R. (2015). Evaluation of
the ESA CCI soil moisture product using ground-based observations. Remote
Sensing of Environment; 162, 380-395, doi: 10.1016/j.rse.2014.07.023.
> 

The "active" and "passive" products were created by using scatterometer and radiometer soil moisture products, respectively. The "combined" product results from a blend based on the two previous products.

This dataset is produced on behalf of the Copernicus Climate Change Service (C3S).